# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("../output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,ribeira grande,38.5167,-28.7000,17.65,100,75,9.26,PT,1682230375
1,1,port elizabeth,-33.9180,25.5701,15.73,95,100,1.15,ZA,1682230208
2,2,papatowai,-46.5619,169.4708,13.12,69,77,11.83,NZ,1682230376
3,3,blackmans bay,-43.0167,147.3167,15.97,58,91,2.14,AU,1682230376
4,4,bethel,41.3712,-73.4140,12.65,90,100,2.57,US,1682230377
...,...,...,...,...,...,...,...,...,...,...
586,586,khandyga,62.6667,135.6000,0.60,77,100,1.89,RU,1682230499
587,587,moriki,12.8741,6.4875,28.35,43,1,4.52,NG,1682230500
588,588,aguas vermelhas,-15.7472,-41.4600,18.15,96,84,2.43,BR,1682230500
589,589,bloemhof,-27.6469,25.6070,15.76,17,7,1.31,ZA,1682230500


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:

# Configure the map plot
city_data_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color = "City")

# Display the map
city_data_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df=city_data_df.loc[(city_data_df["Max Temp"]>=21) & (city_data_df["Max Temp"]<=27)
                               & (city_data_df["Wind Speed"]<4.5)]

# Drop any rows with null values
city_data_df=city_data_df.dropna(how='all')

# Display sample data
#print(city_data_df)
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
18,18,la'ie,21.6477,-157.9253,25.98,77,62,2.24,US,1682230379
20,20,san pedro de uraba,8.2752,-76.3764,22.99,90,100,0.76,CO,1682230379
22,22,three points,32.0767,-111.3137,22.49,19,0,4.12,US,1682230380
31,31,pimenta bueno,-11.6725,-61.1936,21.86,99,100,0.78,BR,1682230383
44,44,taro,-8.3862,115.2800,27.00,69,48,1.52,ID,1682230385
...,...,...,...,...,...,...,...,...,...,...
567,567,san felipe,10.3399,-68.7425,22.71,97,99,0.48,VE,1682230496
568,568,eirunepe,-6.6603,-69.8736,22.29,99,100,0.64,BR,1682230496
572,572,vahdat,38.5563,69.0135,24.41,22,47,2.00,TJ,1682230497
574,574,bahir dar,11.5936,37.3908,25.92,21,34,1.65,ET,1682230497


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df= city_data_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API

hotel_df["Hotel Name"]= " "

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
18,la'ie,US,21.6477,-157.9253,77,
20,san pedro de uraba,CO,8.2752,-76.3764,90,
22,three points,US,32.0767,-111.3137,19,
31,pimenta bueno,BR,-11.6725,-61.1936,99,
44,taro,ID,-8.3862,115.2800,69,
...,...,...,...,...,...,...
567,san felipe,VE,10.3399,-68.7425,97,
568,eirunepe,BR,-6.6603,-69.8736,99,
572,vahdat,TJ,38.5563,69.0135,22,
574,bahir dar,ET,11.5936,37.3908,21,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "format":"json"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    #print(name_address)
    
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
   
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
la'ie - nearest hotel: Courtyard by Marriott
san pedro de uraba - nearest hotel: No hotel found
three points - nearest hotel: No hotel found
pimenta bueno - nearest hotel: Cristalina palace hotel
taro - nearest hotel: Padma Resort Ubud
vila velha - nearest hotel: Hotel Prainha
margaret river - nearest hotel: Margaret River Hotel
port augusta - nearest hotel: Hotel Commonwealth
hawaiian paradise park - nearest hotel: No hotel found
toliara - nearest hotel: Ambary
puerto ayora - nearest hotel: Hostal La Mirada De Solitario George
wailua homesteads - nearest hotel: Hilton Garden Inn Kauai Wailua Bay
hirara - nearest hotel: 古謝そば屋
bassar - nearest hotel: Hôtel Bassar
nkove - nearest hotel: No hotel found
avarua - nearest hotel: Paradise Inn
pisco - nearest hotel: La Portada
codajas - nearest hotel: No hotel found
tolanaro - nearest hotel: Hôtel Mahavokey
san ignacio - nearest hotel: Rainforest Haven Inn
sao felix do xingu - nearest hotel: Hotel Terraço
aripuana - neare

,City,Country,Lat,Lng,Humidity,Hotel Name
18,la'ie,US,21.6477,-157.9253,77,Courtyard by Marriott
20,san pedro de uraba,CO,8.2752,-76.3764,90,No hotel found
22,three points,US,32.0767,-111.3137,19,No hotel found
31,pimenta bueno,BR,-11.6725,-61.1936,99,Cristalina palace hotel
44,taro,ID,-8.3862,115.2800,69,Padma Resort Ubud
...,...,...,...,...,...,...
567,san felipe,VE,10.3399,-68.7425,97,No hotel found
568,eirunepe,BR,-6.6603,-69.8736,99,No hotel found
572,vahdat,TJ,38.5563,69.0135,22,Спитамен-44
574,bahir dar,ET,11.5936,37.3908,21,ናኪ ሆቴል


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [9]:
# Configure the map plot
hotel_df_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color = "City",
    hover_cols = ["Hotel Name", "Country"])
     

# Display the map
hotel_df_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)